In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
train = pd.read_csv('./data/wearly_data/wear_train.csv',index_col=0) #train 0.9
test = pd.read_csv('./data/wearly_data/wear_test.csv',index_col=0) #test 0.1
wear_rate = pd.read_csv('./data/wearly_data/wear_rate.csv',index_col = 0) #total
wear_item = pd.read_csv('./data/wearly_data/wear_item.csv',index_col = 0) #item
wear_user = pd.read_csv('./data/wearly_data/wear_user.csv',index_col=0) #user

In [ ]:
df_train, df_test ,rate

In [9]:
wear_rate

,user,image_file_name,rate,image_id
0,262,https://wearlyimages.s3.amazonaws.com/wearly/m...,2,4148
1,285,https://wearlyimages.s3.amazonaws.com/wearly/m...,3,4148
2,130,https://wearlyimages.s3.amazonaws.com/wearly/m...,2,4148
3,11,https://wearlyimages.s3.amazonaws.com/wearly/m...,1,4148
4,195,https://wearlyimages.s3.amazonaws.com/wearly/m...,3,4148
...,...,...,...,...
30585,101,https://wearlyimages.s3.amazonaws.com/wearly/s...,1,6047
30586,28,https://wearlyimages.s3.amazonaws.com/wearly/o...,2,4928
30587,237,https://wearlyimages.s3.amazonaws.com/wearly/f...,3,3162
30588,11,https://wearlyimages.s3.amazonaws.com/wearly/l...,1,4057


In [57]:
wear_rate.image_file_name.nunique()

7493

In [11]:
USER_NUM = wear_user.user.nunique() #307개
ITEM_NUM = wear_item.image_file_name.nunique() #7628개

AdjacencyUsers = np.zeros((USER_NUM,ITEM_NUM), dtype=np.float32) # N x M shape의 zero matrix 생성 (Adjacency)
DegreeUsers = np.zeros((USER_NUM,1), dtype=np.float32) #N x 1  shape의 zero vactor 생성 (Degree)

AdjacencyItems = np.zeros((ITEM_NUM,USER_NUM), dtype=np.float32) # M x N shape의 zero matrix 생성
DegreeItems =  np.zeros((ITEM_NUM,1), dtype=np.float32)  # M X 1 shape의 zero vactor 생성

for index, row in wear_rate.iterrows():
    userid=int(row['user'])
    itemid=int(row['image_id'])
    AdjacencyUsers[userid][itemid]=row['rate']
    AdjacencyItems[itemid][userid]=row['rate']
    DegreeUsers[userid][0]+=1
    DegreeItems[itemid][0]+=1
    
rating_df = pd.DataFrame(AdjacencyUsers)

In [14]:
rating_df

,0,1,2,3,4,5,6,7,8,9,...,7618,7619,7620,7621,7622,7623,7624,7625,7626,7627
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
302,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
303,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
304,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
305,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Matrix Factorization

In [16]:
class MatrixFactorization():
    def __init__(self, R, k, learning_rate, reg_param, epochs, verbose=False):
        """
        :param R: rating matrix
        :param k: latent parameter
        :param learning_rate: alpha on weight update
        :param reg_param: beta on weight update
        :param epochs: training epochs
        :param verbose: print status
        """

        self._R = R
        self._num_users, self._num_items = R.shape
        self._k = k
        self._learning_rate = learning_rate
        self._reg_param = reg_param
        self._epochs = epochs
        self._verbose = verbose
        
        

    def fit(self):
        """
        training Matrix Factorization : Update matrix latent weight and bias

        참고: self._b에 대한 설명
        - global bias: input R에서 평가가 매겨진 rating의 평균값을 global bias로 사용
        - 정규화 기능. 최종 rating에 음수가 들어가는 것 대신 latent feature에 음수가 포함되도록 해줌.

        :return: training_process
        """

        # init latent features
        self._P = np.random.normal(size=(self._num_users, self._k)) #행렬 P
        self._Q = np.random.normal(size=(self._num_items, self._k)) #행렬 Q

        # init biases
        self._b_P = np.zeros(self._num_users)
        self._b_Q = np.zeros(self._num_items)
        self._b = np.mean(self._R[np.where(self._R != 0)]) #평가가 된 값들의 평균값을 사용

        # train while epochs
        self._training_process = []
        for epoch in range(self._epochs):

            # rating이 존재하는 index를 기준으로 training
            for i in range(self._num_users):
                for j in range(self._num_items):
                    if self._R[i, j] > 0:
                        self.gradient_descent(i, j, self._R[i, j]) #rating이 존재하는경우만
            cost = self.cost()
            self._training_process.append((epoch, cost))

            # print status
            if self._verbose == True and ((epoch + 1) % 10 == 0):
                print("Iteration: %d ; cost = %.4f" % (epoch + 1, cost))


    def cost(self):
        """
        compute root mean square error
        :return: rmse cost
        """

        # xi, yi: R[xi, yi]는 nonzero인 value를 의미한다.
        # 참고: http://codepractice.tistory.com/90
        xi, yi = self._R.nonzero()
        predicted = self.get_complete_matrix()
        cost = 0
        for x, y in zip(xi, yi):
            cost += pow(self._R[x, y] - predicted[x, y], 2)
        return np.sqrt(cost / len(xi))

    def gradient(self, error, i, j):
        """
        gradient of latent feature for GD

        :param error: rating - prediction error
        :param i: user index
        :param j: item index
        :return: gradient of latent feature tuple
        """

        dp = (error * self._Q[j, :]) - (self._reg_param * self._P[i, :])
        dq = (error * self._P[i, :]) - (self._reg_param * self._Q[j, :])
        return dp, dq


    def gradient_descent(self, i, j, rating):
        """
        graident descent function

        :param i: user index of matrix
        :param j: item index of matrix
        :param rating: rating of (i,j)
        """

        # get error
        prediction = self.get_prediction(i, j)
        error = rating - prediction

        # update biases
        self._b_P[i] += self._learning_rate * (error - self._reg_param * self._b_P[i])
        self._b_Q[j] += self._learning_rate * (error - self._reg_param * self._b_Q[j])

        # update latent feature
        dp, dq = self.gradient(error, i, j)
        self._P[i, :] += self._learning_rate * dp
        self._Q[j, :] += self._learning_rate * dq


    def get_prediction(self, i, j):
        """
        get predicted rating: user_i, item_j
        :return: prediction of r_ij
        """
        return self._b + self._b_P[i] + self._b_Q[j] + self._P[i, :].dot(self._Q[j, :].T)


    def get_complete_matrix(self):
        """
        computer complete matrix PXQ + P.bias + Q.bias + global bias

        - PXQ 행렬에 b_P[:, np.newaxis]를 더하는 것은 각 열마다 bias를 더해주는 것
        - b_Q[np.newaxis:, ]를 더하는 것은 각 행마다 bias를 더해주는 것
        - b를 더하는 것은 각 element마다 bias를 더해주는 것

        - newaxis: 차원을 추가해줌. 1차원인 Latent들로 2차원의 R에 행/열 단위 연산을 해주기위해 차원을 추가하는 것.

        :return: complete matrix R^
        """
        return self._b + self._b_P[:, np.newaxis] + self._b_Q[np.newaxis:, ] + self._P.dot(self._Q.T)


    def print_results(self):
        """
        print fit results
        """

        print("User Latent P:")
        print(self._P)
        print("Item Latent Q:")
        print(self._Q.T)
        print("P x Q:")
        print(self._P.dot(self._Q.T))
        print("bias:")
        print(self._b)
        print("User Latent bias:")
        print(self._b_P)
        print("Item Latent bias:")
        print(self._b_Q)
        print("Final R matrix:")
        print(self.get_complete_matrix())
        print("Final RMSE:")
        print(self._training_process[self._epochs-1][1])



## Train

In [17]:
# run example
if __name__ == "__main__":
    # rating matrix - User X Item : 307 X 762)

    # P, Q is (307 X k), (k X 7628) matrix
    factorizer = MatrixFactorization(AdjacencyUsers, k=5, learning_rate=0.01, reg_param=0.01, epochs=50, verbose=True)
    factorizer.fit()
    factorizer.print_results()
    final_df = factorizer.get_complete_matrix() #변환된 R_u,i matrix
    

Iteration: 10 ; cost = 0.5798
Iteration: 20 ; cost = 0.5456
Iteration: 30 ; cost = 0.5284
Iteration: 40 ; cost = 0.5184
Iteration: 50 ; cost = 0.5117
User Latent P:
[[ 6.98514013e-02  3.58490180e-02 -5.74746523e-02 -1.94961721e-01
   1.07354756e-01]
 [ 2.92582778e-02  8.72866870e-02  9.70787900e-05  2.93261717e-03
  -1.13806732e-01]
 [-4.53420142e-03 -2.11186626e-02 -2.88798311e-03 -8.90915132e-02
   7.19052992e-02]
 ...
 [ 2.02941461e-01 -1.02664196e-01  1.98127457e-02  1.02772657e-01
  -9.71017093e-02]
 [-1.23634926e-01 -4.53685410e-02  1.15346709e-01 -6.65801930e-03
  -1.21521324e-03]
 [ 2.70496719e-02 -9.12858764e-02 -2.81961028e-02 -8.63376599e-02
   7.77348991e-02]]
Item Latent Q:
[[ 0.79871368  0.09697732 -2.30129734 ... -0.22610661 -1.83180982
  -0.61081322]
 [-0.51529981  1.20465008 -0.33877559 ...  0.57146258  0.95431806
  -0.77407098]
 [ 0.12403989  1.48630274 -1.50858852 ... -0.18827175 -0.81728422
   1.26340056]
 [-1.52870512 -1.06350638  0.45183094 ...  0.83703193 -0.3555

## RMSE

In [18]:
def get_RMSE(df1,df2):
    raw = df1
    pred = df2
    
    raw[raw ==0] = 'nan'
    
    return np.sqrt(np.nansum((raw - pred)**2 / np.isfinite(raw).sum()))
    
# train_df
get_RMSE(AdjacencyUsers,final_df)

0.5117374288527157

In [21]:
#test_df
get_RMSE(AdjacencyUsers,final_df)

0.5117374288527157

In [20]:
final_df

array([[2.30536479, 1.85097404, 1.94956701, ..., 1.78650187, 1.68872682,
        1.39504226],
       [2.37299522, 1.95245722, 2.13339238, ..., 2.45588836, 2.10768903,
        2.01163006],
       [2.43246326, 2.01525458, 2.36700757, ..., 2.17000153, 1.99915056,
        1.9409884 ],
       ...,
       [2.51823452, 1.74156864, 1.89614833, ..., 2.44265597, 1.61614302,
        2.2084233 ],
       [2.34649178, 2.08672409, 2.51150128, ..., 2.35569933, 2.17326913,
        2.34738639],
       [2.32959151, 1.74302438, 2.20895916, ..., 1.9715006 , 1.7373689 ,
        1.78924677]])